In [2]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import csv
import ast

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [ ]:
df = pd.read_csv('./SpeciesInteractions_EID2.csv')
sorted_df = df.sort_values(by='Carrier')
i = 0
current_string = ''
carrier_id = []
for idx, row in sorted_df.iterrows():
    
    if current_string != row['Carrier']:
        i+=1
        current_string = row['Carrier'] 
    carrier_id.append(i)
sorted_df['Carrier_id'] = carrier_id
sorted_df

In [ ]:
df_grouped_carrier = sorted_df.groupby(['Carrier_id', 'Carrier', 'Carrier classification']).agg({'Cargo' : lambda x: ','.join(x)})
df_grouped_carrier['Cargo'] = df_grouped_carrier['Cargo'].str.split(',')
df_grouped_carrier.to_csv('processed_vertices.csv')
df_grouped_carrier

In [ ]:
df_grouped_cargo = sorted_df.groupby(['Cargo']).agg({'Carrier_id' : lambda x: x.tolist()})
#df_grouped_cargo.to_csv('processed_edges.csv')
df_grouped_cargo = df_grouped_cargo.sort_values(by='Carrier_id')
df_grouped_cargo


In [ ]:
# Leitura e do arquivo e construção do grafo

shared_pathogens = Graph(directed=False) # grafo não-dirigido
shared_pathogens.add_vp('species')
shared_pathogens.add_vp('classification')
shared_pathogens.add_ep('strength')

with open('./processed_vertices.csv') as file:
    vertex_data = csv.reader(file)
    next(vertex_data, None)
    prev_i = -1
    for row in vertex_data:
        i = int(row[0])
        if i != prev_i:
            shared_pathogens.add_vertex(i-1)
        v = shared_pathogens.get_vertex(i-1)
        v['species'] = row[1]
        v['classification'] = row[2]
        prev_i = i       
        
print('número de vértices de g:', shared_pathogens.num_vertices())

In [ ]:
with open('./processed_edges.csv') as file:
    edge_data = csv.reader(file)
    next(edge_data, None)
    for row in edge_data:
        connections = ast.literal_eval(row[1])
        for i in range(len(connections)):
            if i != len(connections)-1:
                next_vertex = i+1
                while(next_vertex < len(connections)):
                    if(connections[i] != connections[next_vertex]):
                        try:
                            shared_pathogens.add_edge(connections[i]-1, connections[next_vertex]-1)
                            shared_pathogens.get_edge(connections[i]-1, connections[next_vertex]-1)['strength'] = 1
                        except ValueError:
                            shared_pathogens.get_edge(connections[i]-1, connections[next_vertex]-1)['strength'] += 1
                        finally:    
                            next_vertex += 1
                    else:
                        next_vertex += 1
                    
                    
print('número de arestas de g:', shared_pathogens.num_edges())

In [ ]:
gt_save(shared_pathogens, 'shared_pathogens.net.gz')

In [ ]:
netpixi.render('shared_pathogens.net.gz');

In [3]:
g = gt_load('shared_pathogens.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)

In [4]:
dc = gt_total_degree(g)
g.add_vp('degree', dc)
gt_save(g, 'test.net.gz')

In [5]:
r = netpixi.render('test.net.gz', infinite=True);

In [17]:
r.edge_scale('strength', 0.01, 30)
r.vertex_scale('degree', 1, 50)
r.vertex_default(color=0x00ff00)
r.edge_default(curve1=1, curve2=1)


In [13]:
r.vertex_set_key('classification')